# Setup

In [1]:
import os
import time
import json
import boto3
import logging
from pathlib import Path
from langchain_aws import ChatBedrockConverse

ModuleNotFoundError: No module named 'langchain_aws'

In [ ]:
# Setup logging
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
from colorama import init, Fore, Style
import logging

# Initialize colorama
init()

# Create a custom formatter
class ColoredFormatter(logging.Formatter):
    def format(self, record):
        message = record.msg
        if isinstance(message, list):
            # Process each message in the list
            formatted_messages = []
            for msg in message:
                if msg.__class__.__name__ == 'HumanMessage':
                    formatted_msg = f"{Fore.GREEN}[Human] {msg.content}{Style.RESET_ALL}"
                elif msg.__class__.__name__ == 'AIMessage':
                    formatted_msg = f"{Fore.BLUE}[AI] {msg.content}{Style.RESET_ALL}"
                elif msg.__class__.__name__ == 'ToolMessage':
                    formatted_msg = f"{Fore.YELLOW}[Tool] {msg.content}...{Style.RESET_ALL}"
                else:
                    formatted_msg = str(msg)
                formatted_messages.append(formatted_msg)
            record.msg = '\n'.join(formatted_messages)
        return super().format(record)
# Set up logger with the custom formatter
logger = logging.getLogger(__name__)
handler = logging.StreamHandler()
handler.setFormatter(ColoredFormatter('%(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)
os.environ["AWS_REGION_NAME"] = "us-east-1"

In [ ]:
ROLE_TO_ASSUME = Path(os.path.join(os.environ["HOME"],"BedrockCrossAccount.txt")).read_text().strip()
logger.info(f"ROLE_TO_ASSUME={ROLE_TO_ASSUME}")
role_to_assume = ROLE_TO_ASSUME
import boto3
import datetime
from botocore.session import get_session
from botocore.credentials import RefreshableCredentials

# ARN of Role A to assume  
role_to_assume = ROLE_TO_ASSUME

def get_credentials():
    sts_client = boto3.client('sts')
    assumed_role = sts_client.assume_role(
        RoleArn=role_to_assume,
        RoleSessionName='cross-account-session',
        # Don't set DurationSeconds when role chaining
    )
    return {
        'access_key': assumed_role['Credentials']['AccessKeyId'],
        'secret_key': assumed_role['Credentials']['SecretAccessKey'],
        'token': assumed_role['Credentials']['SessionToken'],
        'expiry_time': assumed_role['Credentials']['Expiration'].isoformat()
    }

session = get_session()
refresh_creds = RefreshableCredentials.create_from_metadata(
    metadata=get_credentials(),
    refresh_using=get_credentials,
    method='sts-assume-role'
)

# Create a new session with refreshable credentials
session._credentials = refresh_creds
boto3_session = boto3.Session(botocore_session=session)

In [ ]:
ROLE_TO_ASSUME = Path(os.path.join(os.environ["HOME"],"BedrockCrossAccount.txt")).read_text().strip()
logger.info(f"ROLE_TO_ASSUME={ROLE_TO_ASSUME}")
role_to_assume = ROLE_TO_ASSUME
import boto3
import datetime
from botocore.session import get_session
from botocore.credentials import RefreshableCredentials

# ARN of Role A to assume  
role_to_assume = ROLE_TO_ASSUME

def get_credentials():
    sts_client = boto3.client('sts')
    assumed_role = sts_client.assume_role(
        RoleArn=role_to_assume,
        RoleSessionName='cross-account-session',
        # Don't set DurationSeconds when role chaining
    )
    return {
        'access_key': assumed_role['Credentials']['AccessKeyId'],
        'secret_key': assumed_role['Credentials']['SecretAccessKey'],
        'token': assumed_role['Credentials']['SessionToken'],
        'expiry_time': assumed_role['Credentials']['Expiration'].isoformat()
    }

session = get_session()
refresh_creds = RefreshableCredentials.create_from_metadata(
    metadata=get_credentials(),
    refresh_using=get_credentials,
    method='sts-assume-role'
)

# Create a new session with refreshable credentials
session._credentials = refresh_creds
boto3_session = boto3.Session(botocore_session=session)
# ---- ⚠️ Update region for your AWS setup ⚠️ ----
bedrock_client = boto3_session.client("bedrock-runtime",
                              region_name=region)

In [ ]:
region: str = "us-east-1"
from langchain_aws import ChatBedrockConverse
import boto3

# ---- ⚠️ Update region for your AWS setup ⚠️ ----
bedrock_client = boto3_session.client("bedrock-runtime",
                              region_name=region)

# Data Processing & Storage